In [1]:
import pathlib
import random
import string
import re
import io

import numpy as np
import pandas as pd

from tensorflow import data as tf_data
from tensorflow import strings as tf_strings

import keras
from keras import ops
from keras import layers
from keras.layers import TextVectorization

2024-12-25 12:24:48.675495: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
path = pathlib.Path("merged_output.txt")
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

max_size = 500_000
text = text[:max_size]

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 13464701
Total chars: 87
Number of sequences: 166654


In [60]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128, return_sequences=True),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [62]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [64]:
epochs = 10
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % (epoch+1))

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print("-")

1302/1302 ━━━━━━━━━━━━━━━━━━━━ 146s 111ms/step - loss: 2.8159

Generating text after epoch: 1
...Diversity: 0.2
...Generating with seed: "ію. згоден, порушив, а чому — не знаю. н"
...Generated:  а відній покради постальні почали відна відна заленти постальні постальні відна залетим у світні навідка завидили відна відна відна завдали світ почали вийдали відна відна віднія на наста відна постальний постальний на вийдині відна відна посом відна відна відна завжю на навіть до відна віднія відна на відна відна відна відна простальний відна на навіть на відно відна відно на постальні постальний
-
...Diversity: 0.5
...Generating with seed: "ію. згоден, порушив, а чому — не знаю. н"
...Generated:  ас коль у викали відна прокодав політь і не відкали світом самі завдинали бульом заподиналі відний залюває мати на встом завіднія прочіза так скажаль на відивали вій устела буль від не виколів почолі попокав запового знаєть у не растом — мені почорую, посулів. він пареди відній захоліть, а віднія, ко

KeyboardInterrupt: 

In [66]:
model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_21 (LSTM)                  │ (None, 40, 128)        │       110,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_22 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 87)             │        11,223 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 506,800 (1.93 MB)

 Trainable params: 253,399 (989.84 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 253,401 (989.85 KB)